In [5]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
seed = 111
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

text = data.Field(tokenize = 'spacy' , tokenizer_language = 'en_core_web_sm')
label = data.LabelField()

train_data, test_data = datasets.TREC.splits(text, label, fine_grained=False)

train_data, valid_data = train_data.split(random_state = random.seed(seed))

In [8]:
vars(train_data[-1])

{'text': ['How',
  'many',
  'rows',
  'of',
  'sprocket',
  'holes',
  'does',
  'a',
  'roll',
  'of',
  '35',
  '-',
  'millimeter',
  'film',
  'have',
  '?'],
 'label': 'NUM'}

In [10]:
MAX_VOCAB_SIZE = 25_000

text.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

label.build_vocab(train_data)

In [19]:
# set the iterator 
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator , valid_iterator , test_iterator = data.BucketIterator.splits(
(train_data , valid_data, test_data) , 
batch_size = batch_size, 
    device = device
)

In [17]:
vars(label.vocab)

{'freqs': Counter({'ABBR': 64,
          'ENTY': 868,
          'HUM': 854,
          'DESC': 803,
          'LOC': 596,
          'NUM': 631}),
 'itos': ['ENTY', 'HUM', 'DESC', 'NUM', 'LOC', 'ABBR'],
 'unk_index': None,
 'stoi': defaultdict(None,
             {'ENTY': 0, 'HUM': 1, 'DESC': 2, 'NUM': 3, 'LOC': 4, 'ABBR': 5}),
 'vectors': None}

In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size , embedding_dim, dropout, pad_idx, 
                output_dim, n_filters, kernels):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim, padding_idx = pad_idx)
        self.conv = nn.ModuleList([nn.Conv2d(in_channels = 1,
                                            out_channels = n_filters , 
                                            kernel_size = (fs , embedding_dim)) for fs in kernels])
        self.fc = nn.Linear(len(kernels)*n_filters, output_dim )
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        
        x = x.permute(1 , 0)
        # [batch size , sen len ]
        x = self.embedding(x)
        # [ batch size , sen len , embedding dim ]
        x = x.unsqueeze(1)
        # [batch size , 1 , sen len , embedding dim]
        x = [F.relu(f(x)).squeeze(3) for f in self.conv]
        # [ batch size , n_filters , sen len -n_kernel_size +1]
        pooled = [F.max_pool1d(conv , conv.shape[2]).squeeze(2) for conv in x]
        #pooled = [batch size , n_filters]
        
        cat = self.dropout(torch.cat(pooled , dim=1))
        #return [batch size , n_filters*len(filters_size)]
        return self.fc(cat)
    

In [83]:
input_dim = len(text.vocab)
output_dim = 10
kernel_sizes = [2,3,4]
embedding_dim = 100
n_filters = 80
dropout = 0.4
pad_idx = text.vocab.stoi[text.pad_token]
model =CNN(input_dim, embedding_dim, dropout, pad_idx, output_dim, 
                 n_filters, kernel_sizes)
#  vocab_size , embedding_dim, dropout, pad_idx, 
#                 output_dim, n_filters, kernels)

In [84]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 821,650 trainable parameters


In [85]:
#load the pre_train embedding into the model
pretrained_embeddings = text.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.2634, -0.2391,  0.3198,  ...,  1.5451, -1.3952,  1.9205],
        [ 1.0707, -0.6284, -0.8112,  ...,  0.9490,  0.5720,  1.2499],
        [ 0.1638,  0.6046,  1.0789,  ..., -0.3140,  0.1844,  0.3624],
        ...,
        [-0.6763,  0.1671,  0.9469,  ..., -0.3254, -0.2345,  0.2310],
        [-0.0515, -0.3798, -0.5699,  ..., -0.0349,  0.3298,  1.1395],
        [ 0.0091,  0.2810,  0.7356,  ..., -0.7508,  0.8967, -0.7631]])

In [86]:
UNK_IDX = text.vocab.stoi[text.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)

In [87]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

In [88]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [89]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [90]:
def train(model , train_iterator , optimizer , criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for batch in train_iterator:
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions , batch.label)
        acc = categorical_accuracy(predictions , batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss +=loss.item()
        epoch_acc += acc.item()
    return epoch_loss/len(train_iterator) , epoch_acc/len(train_iterator)


In [91]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [92]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [93]:
n_epochs = 5
best_valid_loss = float('inf')
for epoch in range(n_epochs):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 1.338 | Train Acc: 47.59%
	 Val. Loss: 0.941 |  Val. Acc: 67.82%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.859 | Train Acc: 69.90%
	 Val. Loss: 0.765 |  Val. Acc: 72.09%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.637 | Train Acc: 78.15%
	 Val. Loss: 0.635 |  Val. Acc: 76.00%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.482 | Train Acc: 84.38%
	 Val. Loss: 0.567 |  Val. Acc: 79.65%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.362 | Train Acc: 89.17%
	 Val. Loss: 0.508 |  Val. Acc: 81.01%


In [94]:
model.load_state_dict(torch.load('tut5-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.405 | Test Acc: 85.73%


In [102]:
import spacy
nlp = spacy.load('en_core_web_sm')
def predict_class(model , sentence , min_len = 4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>']*(min_len-len(tokenized))
    indexed = [text.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
#     preds = model.unsqueeze(1)
    preds = model(tensor)
    max_preds =  preds.argmax(dim = 1)
    return max_preds.item()

In [105]:
pred_class = predict_class(model, "where can i find the house?")
print(f'Predicted class is: {pred_class} = {label.vocab.itos[pred_class]}')

Predicted class is: 2 = DESC


In [101]:
sentence = ['i have a fish !']
tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
if len(tokenized) < min_len:
    tokenized += ['<pad>']*(min_len-len(tokenized))
indexed = [text.vocab.stoi[t] for t in tokenized]
tensor = torch.LongTensor(indexed).to(device)
tensor = tensor.unsqueeze(1)

988